In [ ]:
import boto3

In [ ]:
cf_client = boto3.client('cloudformation')

In [ ]:
cf_client.list_stacks()

In [ ]:
stack_desc = cf_client.describe_stacks(
    StackName='FifoStack'
)

outputs = stack_desc['Stacks'][0]['Outputs']
print(outputs)

In [ ]:
topic1Arn = [o['OutputValue'] for o in outputs if o['OutputKey'] == 'fifoTopic1Arn'][0] 
print(topic1Arn)
topic2Arn = [o['OutputValue'] for o in outputs if o['OutputKey'] == 'fifoTopic2Arn'][0] 
print(topic2Arn)
s1QueueUrl = [o['OutputValue'] for o in outputs if o['OutputKey'] == 's1QueueUrl'][0] 
print(s1QueueUrl)

In [ ]:
sns_client = boto3.client('sns')
sqs_client = boto3.client('sqs')

In [ ]:
sns_client.publish(
    TopicArn=topic1Arn,
    Message='{"this":false}',
    MessageGroupId='xx'
)

In [ ]:
sns_client.publish(
    TopicArn=topic2Arn,
    Message='{"another":true}',
    MessageGroupId='xx'
)

In [ ]:
sqs_response = sqs_client.receive_message(
    QueueUrl=s1QueueUrl
)
print(sqs_response)

if 'Messages'in sqs_response:
    receiptHandle = sqs_response['Messages'][0]['ReceiptHandle']
    print('delete message with handle {}'.format(receiptHandle))
    sqs_client.delete_message(
        QueueUrl=s1QueueUrl,
        ReceiptHandle=sqs_response['Messages'][0]['ReceiptHandle']
    )